<a href="https://colab.research.google.com/github/e19166/e19-4yp-Dynamic-Multi-Dimensional-Resource-Orchestration-in-Kubernetes/blob/main/service1_cpu_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('service1_cpu_limit_reduction.csv')
df.head()  # to preview the data

,Timestamp,Service,CPU Request,Memory Request,CPU Limit,Memory Limit,Latency,CPU Usage,Memory Usage
0,2025-05-11T14:10:23.603000,service-1-deployment,0.1,134217728,0.3,268435456,0.003685,0.186381,185323520
1,2025-05-11T14:10:53.686000,service-1-deployment,0.1,134217728,0.3,268435456,0.003676,0.183662,185323520
2,2025-05-11T14:11:23.750000,service-1-deployment,0.1,134217728,0.3,268435456,0.003676,0.182106,185323520
3,2025-05-11T14:11:53.827000,service-1-deployment,0.1,134217728,0.3,268435456,0.003742,0.189612,185323520
4,2025-05-11T14:12:23.909000,service-1-deployment,0.1,134217728,0.3,268435456,0.003765,0.190540,185323520
